In [1]:
import sys
import numpy as np
import torch
import learn2learn as l2l
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from PIL.Image import LANCZOS
from PIL import Image

In [2]:
torch.cuda.is_available()

True

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.10.1101450599/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.10.1101450599/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [4]:
from data.taskers import gen_tasks
from src.zoo.archs import CCVAE, CEncoder, Classifier_VAE, ConvBase, TADCEncoder, ResNet12Backbone, WRN28Backbone, CDecoder
from src.zoo.delpo_utils import setup, inner_adapt_delpo, loss, accuracy

In [5]:
train_tasks = gen_tasks(dataname='miniimagenet', root='../../dataset/mini_imagenet', mode='train', n_ways=5, k_shots=1, q_shots=5, image_transforms=None)

In [6]:
device = 'cpu'
n_ways = 5
k_shots = 1
q_shots = 5
dataset = 'miniimagenet'
root = '../../dataset/mini_imagenet'
order = False
inner_lr = 0.001
meta_lr = 0.0001
reconst_loss = nn.MSELoss(reduction='none')
adapt_steps = 2
meta_batch_size = 10
download = False

In [7]:
class Args:
    def __init__(self, wt_ce, klwt, rec_wt, beta_l, beta_s):
        #args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s
        self.wt_ce = wt_ce
        self.klwt = klwt
        self.rec_wt = rec_wt
        self.beta_l = beta_l
        self.beta_s = beta_s
        self.dataset = 'miniimagenet'
        self.task_adapt = True
        self.task_adapt_fn = 'eaen'
        self.n_ways = 5
        self.k_shots = 1
        self.q_shots = 5
        self.pretrained = [True, '../../dataset/backbones/mini/25u1gtp5.pth', 16000]
args = Args(100, False, 0.01, 1, 1)

In [8]:
learner = CCVAE(in_channels=3, base_channels=32,
                        n_ways=n_ways, dataset='mini_imagenet', task_adapt=args.task_adapt, task_adapt_fn=args.task_adapt_fn, args=args)
learner = learner.to(device)
learner = l2l.algorithms.MAML(learner, first_order=order, lr=inner_lr)

# Init the Backbone
if args.pretrained[0] == True:
    backbone = ResNet12Backbone(
        args, avg_pool=True if args.pretrained[2] == 640 else False)  # F => 16000; T => 640
    weights = torch.load(args.pretrained[1])
    backbone.load_state_dict(weights)
    backbone.to(device)
    # Freeze the backbone
    for p in backbone.parameters():
        p.requires_grad = False
else:
    backbone = 'None'


In [9]:

def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def kl_div(mus, log_vars):
    return - 0.5 * (1 + log_vars - mus**2 - torch.exp(log_vars)).sum(dim=1)


def loss(reconst_loss: object, reconst_image, image, logits, labels, mu_s, log_var_s, mu_l, log_var_l, wt_ce=1e2, klwt=False, rec_wt=1e-2, beta_l=1, beta_s=1):
    kl_div_s = kl_div(mu_s, log_var_s).mean()
    kl_div_l = kl_div(mu_l, log_var_l).mean()
    if klwt:
        kl_wt = mu_s.shape[-1] / (image.shape[-1] *
                                  image.shape[-2] * image.shape[-3])
    else:
        kl_wt = 1

    ce_loss = torch.nn.CrossEntropyLoss()
    classification_loss = ce_loss(logits, labels)
    rec_loss = reconst_loss(reconst_image, image)
    rec_loss = rec_loss.view(rec_loss.shape[0], -1).sum(dim=-1).mean()

    L = wt_ce*classification_loss + beta_l*kl_wt*kl_div_l + \
        rec_wt*rec_loss + beta_s*kl_wt*kl_div_s  # -log p(x,y)

    losses = {'elbo': L, 'label_kl': kl_div_l, 'style_kl': kl_div_s,
              'reconstruction_loss': rec_loss, 'classification_loss': classification_loss}

    return losses


In [10]:
task = train_tasks.sample()

In [11]:
data, labels = task
if args.dataset == 'miniimagenet':
    data, labels = data.to(device) / 255.0, labels.to(device)
elif (args.dataset == 'omniglot') or (args.dataset == 'cifarfs') or (args.dataset == 'tiered'):
    data, labels = data.to(device), labels.to(device)
total = n_ways * (k_shots + q_shots)
queries_index = np.zeros(total)

# Extracting the evaluation datums from the entire task set, for the meta gradient calculation
for offset in range(n_ways):
    queries_index[np.random.choice(
        k_shots+q_shots, q_shots, replace=False) + ((k_shots + q_shots)*offset)] = True
support = data[np.where(queries_index == 0)]
support_labels = labels[np.where(queries_index == 0)]
queries = data[np.where(queries_index == 1)]
queries_labels = labels[np.where(queries_index == 1)]


In [12]:
queries.shape

torch.Size([25, 3, 84, 84])

In [13]:

if args.pretrained[0] == True:
    support_ext = backbone(support)
    queries_ext = backbone(queries)


/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [14]:
support_ext.shape

torch.Size([5, 640])

In [15]:

# Inner adapt step
if args.pretrained[0] == True:
    for _ in range(adapt_steps):
        if args.task_adapt:
            reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner([torch.cat(
                [support_ext, queries_ext], dim=0), torch.cat([support, queries], dim=0)], 'inner')
        else:
            reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
                [support_ext, support], 'inner')
        adapt_loss = loss(reconst_loss, reconst_image, support,
                            logits, support_labels, mu_s, log_var_s, mu_l, log_var_l, args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s)
        learner.adapt(adapt_loss['elbo'])

    if args.task_adapt:
        reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner([torch.cat(
            [support_ext, queries_ext], dim=0), torch.cat([support, queries], dim=0)], 'outer')
    else:
        reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
            [queries_ext, queries], 'outer')

if args.pretrained[0] == False:
    for _ in range(adapt_steps):
        if args.task_adapt:
            reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
                torch.cat([support, queries], dim=0), 'inner')
        else:
            reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
                support, 'inner')
        adapt_loss = loss(reconst_loss, reconst_image, support,
                            logits, support_labels, mu_s, log_var_s, mu_l, log_var_l, args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s)
        learner.adapt(adapt_loss['elbo'])

    if args.task_adapt:
        reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
            torch.cat([support, queries], dim=0), 'outer')
    else:
        reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
            queries, 'outer')

eval_loss = loss(reconst_loss, reconst_image, queries,
                    logits, queries_labels, mu_s, log_var_s, mu_l, log_var_l, args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s)
eval_acc = accuracy(F.softmax(logits, dim=1), queries_labels)


RuntimeError: number of dims don't match in permute

In [7]:
backbone = ResNet12Backbone(args=args, avg_pool=True)
weights = torch.load('../../dataset/backbones/mini/25u1gtp5.pth', map_location='cpu')
backbone.load_state_dict(weights)
backbone.to('cuda')

ResNet12Backbone(
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): LeakyReLU(negative_slope=0.1)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (downsample): Sequential(
        (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (DropBlock): DropBlock()
    )


In [8]:
for p in backbone.parameters():
    p.requires_grad = False

In [10]:
x = backbone(train_tasks.sample()[0].to('cuda'))
x.shape

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([100, 640])

In [21]:
#x.detach()
x.requires_grad

False